In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))
from models.cifar import ResNet

from utils.loading import load_net_cifar

from utils.data import make_generators_DF_cifar, make_generators_DF_MNIST
from utils.loading import vae_from_args
from utils.train_val_auto import validate_epoch_auto
from utils.evaluation import evaluate_adv_files_df, get_metrics

import torch
from torch import nn
from torch.autograd import Variable
import torchvision.transforms as T
import foolbox
import json
from PIL import Image

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.current_device())

cuda:1
0


## Standard PreAct ResNet 18

In [21]:
batch_size = 128
num_workers = 4
model_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/models/preact_resnet-18_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/files_df.pkl'
adv_files_df_loc = '/media/rene/data/adv_consistency/cifar10/adv_fgsm/preact_resnet-18-0.05-1/files_df_adv.pkl'

with open(files_df_loc , 'rb') as f:
    files_df = pickle.load(f)
with open(adv_files_df_loc , 'rb') as f:
    adv_files_df = pickle.load(f)


with torch.cuda.device(1):
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                            path_colname='path', adv_path_colname=None, return_loc=False)
    model = load_net_cifar(model_loc).to(device).eval()
    validate_epoch(dataloaders['val'], model, device)
    
    
    dataloaders = make_generators_DF_cifar(adv_files_df, batch_size, num_workers, size=32, 
                                            path_colname='adv_path', adv_path_colname=None, return_loc=False)
    model = load_net_cifar(model_loc).to(device).eval()
    acc = validate_epoch(dataloaders['val'], model, device)

Loading model_file preact_resnet-18_model_best.pth.tar
VALID:  * TOP1 94.510 TOP5 99.750 Loss (0.0000)	 Time (0.019)	
Loading model_file preact_resnet-18_model_best.pth.tar
VALID:  * TOP1 58.350 TOP5 95.249 Loss (0.0000)	 Time (0.019)	


## PreAct ResNet 18 with topK
* Something going very wrong here

In [36]:
batch_size = 128
num_workers = 4
model_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/models/preact_resnet-18-0.05-1_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/files_df.pkl'
adv_files_df_loc = '/media/rene/data/adv_consistency/cifar10/adv_fgsm/preact_resnet-18-0.05-1/files_df_adv.pkl'

with open(files_df_loc , 'rb') as f:
    files_df = pickle.load(f)
with open(adv_files_df_loc , 'rb') as f:
    adv_files_df = pickle.load(f)


with torch.cuda.device(1):
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                            path_colname='path', adv_path_colname=None, return_loc=False)
    model = load_net_cifar(model_loc).to(device).eval()
    validate_epoch(dataloaders['val'], model, device)


    dataloaders = make_generators_DF_cifar(adv_files_df, batch_size, num_workers, size=32, 
                                            path_colname='adv_path', adv_path_colname=None, return_loc=False)
    model = load_net_cifar(model_loc).to(device).eval()
    acc = validate_epoch(dataloaders['val'], model, device)

Loading model_file preact_resnet-18-0.05-1_model_best.pth.tar
VALID:  * TOP1 80.440 TOP5 98.110 Loss (0.0000)	 Time (0.019)	
Loading model_file preact_resnet-18-0.05-1_model_best.pth.tar
VALID:  * TOP1 35.695 TOP5 85.880 Loss (0.0000)	 Time (0.019)	


## ResNet 18 and 34 with No ReLU and topK instead

In [35]:
batch_size = 128
num_workers = 4
files_df_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/files_df.pkl'

with open(files_df_loc , 'rb') as f:
    files_df = pickle.load(f)

with torch.cuda.device(1):
    model_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/models/PResNetRegNoRelU-18-0.05-1_model_best.pth.tar'
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                            path_colname='path', adv_path_colname=None, return_loc=False)
    model = load_net_cifar(model_loc).to(device).eval()
    validate_epoch(dataloaders['val'], model, device)

    model_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/models/PResNetRegNoRelU-34-0.05-1_model_best.pth.tar'
    model = load_net_cifar(model_loc).to(device).eval()
    acc = validate_epoch(dataloaders['val'], model, device)

Loading model_file PResNetRegNoRelU-18-0.05-1_model_best.pth.tar
VALID:  * TOP1 88.870 TOP5 99.490 Loss (0.0000)	 Time (0.107)	
Loading model_file PResNetRegNoRelU-34-0.05-1_model_best.pth.tar
VALID:  * TOP1 89.540 TOP5 99.590 Loss (0.0000)	 Time (0.170)	


## PreAct ResNet with no ReLU

In [4]:
batch_size = 128
num_workers = 4
model_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/models/PResNetRegNoRelU-34-0.05-1_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_consistency/cifar10/cifar10_normal/files_df.pkl'
adv_files_df_loc = '/media/rene/data/adv_consistency/cifar10/adv_fgsm/PResNetRegNoRelU-34-0.05-1/files_df_adv.pkl'

with open(files_df_loc , 'rb') as f:
    files_df = pickle.load(f)
with open(adv_files_df_loc , 'rb') as f:
    adv_files_df = pickle.load(f)


with torch.cuda.device(1):
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                            path_colname='path', adv_path_colname=None, return_loc=False)
    model = load_net_cifar(model_loc).to(device).eval()
    validate_epoch(dataloaders['val'], model, device)


    dataloaders = make_generators_DF_cifar(adv_files_df, batch_size, num_workers, size=32, 
                                            path_colname='adv_path', adv_path_colname=None, return_loc=False)
    model = load_net_cifar(model_loc).to(device).eval()
    acc = validate_epoch(dataloaders['val'], model, device)

Loading model_file PResNetRegNoRelU-34-0.05-1_model_best.pth.tar
VALID:  * TOP1 89.540 TOP5 99.590 Loss (0.0000)	 Time (0.185)	
Loading model_file PResNetRegNoRelU-34-0.05-1_model_best.pth.tar
VALID:  * TOP1 55.763 TOP5 99.498 Loss (0.0000)	 Time (0.172)	
